In [ ]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

Без использования merge!

In [ ]:
trans = pd.read_csv("/content/drive/MyDrive/Data/transactions.csv").sample(1000, random_state=1)
tr_ty = pd.read_csv("/content/drive/MyDrive/Data/tr_types.csv",sep=";")

In [ ]:
tr_ty[(tr_ty.tr_description.str.find("POS")!=-1) | (tr_ty.tr_description.str.find("ATM")!=-1)].tr_type.unique().shape[0] / trans.tr_type.unique().shape[0]

0.875

## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости.

In [ ]:
top10 = trans["tr_type"].value_counts()[:10]
top10

1010    245
2010    147
1030    143
1110    132
7070     98
2370     64
7010     30
7030     20
1100     17
1200     17
Name: tr_type, dtype: int64

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите разницу для этих клиентов между суммой расходов и суммой приходов

In [ ]:
id_max_cash = trans.groupby(trans[trans.amount > 0].customer_id).sum().amount.idxmax()
max_cash = trans.groupby(trans[trans.amount > 0].customer_id).sum().amount.max()

dif_out_in1 = trans[(trans.amount < 0) & (trans.customer_id == id_max_cash)].amount.sum() - trans[(trans.amount > 0) & (trans.customer_id == id_max_cash)].amount.sum() 


id_max_spend = trans.groupby(trans[trans.amount < 0].customer_id).sum().amount.idxmin()
max_spend = trans.groupby(trans[trans.amount < 0].customer_id).sum().amount.min()

dif_out_in2 = trans[(trans.amount < 0) & (trans.customer_id == id_max_spend)].amount.sum() - trans[(trans.amount > 0) & (trans.customer_id == id_max_spend)].amount.sum() 


print(f"id {id_max_cash} max cash {max_cash}, difference between spend and cash {dif_out_in1}")
print(f"id {id_max_spend} max spend {max_spend}, difference between cash and spend {dif_out_in2}")

id 48433830.0 max cash 1909028.41, difference between spend and cash -1909028.41
id 66647597.0 max spend -4491831.54, difference between cash and spend -4491831.54


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
#1
ten = dict(top10)
for tr in ten.keys():
  ten[tr] = (sum(trans[trans["tr_type"] == tr].amount)/ten[tr],trans[trans["tr_type"] == tr].amount.median())
print(ten)

{1010: (-23887.977346938787, -6715.29), 2010: (-145740.07210884365, -44918.32), 1030: (-3528.2190909090905, -2245.92), 1110: (-30586.12833333333, -10101.005000000001), 7070: (74012.93540816326, 14025.74), 2370: (-146191.55296875, -38180.57), 7010: (315551.1663333333, 145984.525), 7030: (27515.928499999998, 13026.31), 1100: (-45406.37882352941, -11229.58), 1200: (-28233.09, -6715.29)}


In [ ]:
print(f"Median for client max spend {trans[trans.customer_id == id_max_spend].amount.median()}")
print(f"Mean For client max spend {trans[trans.customer_id == id_max_spend].amount.mean()} \n")

print(f"Median for client max cashe {trans[trans.customer_id == id_max_cash].amount.median()}")
print(f"Mean For client max cashe {trans[trans.customer_id == id_max_cash].amount.mean()}")

Median for client max spend -4491831.54
Mean For client max spend -4491831.54 

Median for client max cashe 1909028.41
Mean For client max cashe 1909028.41


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
import pandas as pd

In [ ]:
transactions = pd.read_csv("/content/drive/MyDrive/Data/transactions.csv")
gender_train = pd.read_csv("/content/drive/MyDrive/Data/gender_train.csv")
tr_mcc_codes = pd.read_csv("/content/drive/MyDrive/Data/tr_mcc_codes.csv",sep=';')
tr_types = pd.read_csv("/content/drive/MyDrive/Data/tr_types.csv",sep=';')
transactions.shape

(6849346, 6)

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left', on = "customer_id")
transactions = pd.merge(transactions, tr_mcc_codes, how='inner',on = "mcc_code")
transactions = pd.merge(transactions, tr_types, how='inner',on = "tr_type")
transactions.shape

(6846580, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
abs(transactions[(transactions["gender"] == 0) & (transactions["amount"] < 0)].amount.mean() - transactions[(transactions["gender"] == 1) & (transactions["amount"] < 0)].amount.mean())

24748.03831069907

In [ ]:
abs(transactions[(transactions["gender"] == 0) & (transactions["amount"] > 0)].amount.mean() - transactions[(transactions["gender"] == 1) & (transactions["amount"] > 0)].amount.mean())

65084.173920174886

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 транзакций для мужчин и для женщин, наименьших среди всех транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
max_girl = pd.DataFrame(transactions.groupby(transactions[(transactions.amount > 0) & (transactions.gender == 0)].tr_type).amount.max().sort_values().iloc[:10])
max_girl.rename(columns = {"amount":"amount_girl"}, inplace=True)
max_men = pd.DataFrame(transactions.groupby(transactions[(transactions.amount > 0) & (transactions.gender == 1)].tr_type).amount.max().sort_values().iloc[:10])
max_men.rename(columns = {"amount":"amount_men"}, inplace=True)
max_income = max_men.merge(max_girl,on = "tr_type")
max_income

,amount_men,amount_girl
tr_type,,
4051.0,381.81,1122.96
4071.0,673.77,1010.66
4110.0,3144.28,4469.37
4210.0,4166.17,5246.91
4100.0,5220.63,5041.86
8146.0,22459.16,14823.04


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности для в тратах для разных полов.

In [ ]:
men_spend = transactions.groupby(transactions[(transactions.gender == 1) & (transactions.amount  < 0)].mcc_code).amount.sum()
girl_spend = transactions.groupby(transactions[(transactions.gender == 0) & (transactions.amount  < 0)].mcc_code).amount.sum()
girl_spend.sub(men_spend,).dropna().abs().sort_values().iloc[-5:].index

Float64Index([5812.0, 5541.0, 6010.0, 4829.0, 6011.0], dtype='float64', name='mcc_code')

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount>0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [ ]:
transactions.tr_datetime = transactions.tr_datetime.apply(lambda x:x.split()[-1][:x.split()[-1].find(":")])
trans = transactions[(transactions.amount > 0) & (transactions.tr_datetime == "00")]
print(f"For men {trans[trans.gender == 1].shape[0]}")
print(f"For men {trans[trans.gender == 0].shape[0]}")

For men 10335
For men 8357
